In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/SyNet_bc_filtered_scaled.csv', index_col=[0], header=[0])
df

,0,1,2,3,4,5,6,7,8,9,...,11738,11739,11740,11741,11742,11743,11744,11745,11746,11747
GSM107072;Pawitan,-0.602517,-0.217145,0.281423,0.085491,-0.278764,-0.120272,-0.208145,-0.324611,-0.283117,0.194528,...,-0.199445,-0.252016,-0.001522,0.101132,-0.227703,0.211490,-0.299598,-0.309219,-0.292792,0.018368
GSM107073;Pawitan,0.303390,-0.146267,-0.308157,-0.549286,0.041284,-0.349625,-0.542238,-0.388602,0.146809,-0.064065,...,-0.608274,-0.583912,-0.566102,0.300822,-0.049000,-0.125753,0.352915,-0.659579,-0.699349,0.991560
GSM107074;Pawitan,0.280339,-0.564614,-0.135844,-0.674381,-0.422376,0.045734,-0.705008,0.087699,0.122105,-0.198941,...,-0.105184,-0.983028,0.165270,-0.183011,0.120070,-0.334216,0.133344,-0.359665,-0.384301,-0.412687
GSM107075;Pawitan,-0.086415,-0.459489,-0.077784,-0.133890,-0.563336,0.199338,-0.160014,0.550678,-0.647363,-0.695306,...,-0.336746,-0.223510,-0.383515,-0.112815,-0.321696,-0.896174,-0.047811,-0.212386,0.227248,-0.363852
GSM107076;Pawitan,-0.322869,-0.286967,-0.375086,-0.298586,-0.441541,-0.214382,-0.065838,-0.081867,-0.246255,-0.289959,...,-0.143308,-0.164462,-0.220164,-0.459289,-0.138438,-0.319103,0.180149,-0.376760,-0.568092,-0.061105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TCGA-E2-A108-01;TCGA,0.025287,0.356513,-0.042358,0.218662,-0.352445,-0.196461,0.426815,0.275894,-0.316390,0.445714,...,-0.032373,0.176079,-0.288418,-0.336976,0.228340,-0.185076,-0.348199,-0.158360,0.000730,-0.260105
TCGA-BH-A0B2-01;TCGA,-0.011613,0.215292,-0.034698,0.235126,-0.070703,-0.197445,0.264458,0.064942,-0.413348,-0.022968,...,-0.150001,-0.011208,-0.156629,-0.584883,-0.107509,-0.153384,-0.154076,0.185658,-0.185888,0.036626
TCGA-AO-A12B-01;TCGA,-0.377870,-0.385831,-0.436257,-0.469226,-0.764166,0.016416,0.219710,-0.521436,0.033584,0.164853,...,0.195169,-0.089160,-0.486087,-0.006563,-0.534348,-0.485102,-0.202320,0.210931,0.146989,0.384576
TCGA-E2-A109-01;TCGA,-0.298234,-0.307404,0.230935,-0.301801,-0.005811,0.029275,-0.023483,-0.160973,0.037851,-0.022120,...,0.110209,-0.367736,-0.359516,0.631281,-0.578857,-0.266369,-0.004335,0.056546,-0.326707,0.075211


In [4]:
# Create new df for batch labels
import string

id_list = []
study_list = []
cvset_list = []

# Take the index
sample_list = list(df.index.values)


for name in sample_list:
    split = name.split(';') 
    id_list.append(split[0])
    study_list.append(split[1])
    if split[1] == 'TCGA':
        cvset_list.append('val')
    elif split[1] == 'Symmans':
        cvset_list.append('test')
    elif split[1] == 'Miller' or split[1] == 'Minn':
        cvset_list.append('final_test')
    else:
        cvset_list.append('train')
        
study_name_list = sorted(set(study_list))
# study_name_list = set(study_list)
batch_label_list = list(string.ascii_uppercase[:len(study_name_list)])
study_name_dict = dict(zip(study_name_list, batch_label_list))

if len(id_list) == len(study_list) == len(cvset_list):
    df_batch_label = pd.DataFrame(list(zip(id_list, study_list, cvset_list)), columns=['sample_id', 'study_name', 'cvset'])

df_batch_label['batch_label'] = df_batch_label['study_name'].apply (lambda x: study_name_dict.get(x))
df_batch_label = df_batch_label[['sample_id', 'study_name', 'batch_label', 'cvset']]

df_batch_label

,sample_id,study_name,batch_label,cvset
0,GSM107072,Pawitan,F,train
1,GSM107073,Pawitan,F,train
2,GSM107074,Pawitan,F,train
3,GSM107075,Pawitan,F,train
4,GSM107076,Pawitan,F,train
...,...,...,...,...
3965,TCGA-E2-A108-01,TCGA,I,val
3966,TCGA-BH-A0B2-01,TCGA,I,val
3967,TCGA-AO-A12B-01,TCGA,I,val
3968,TCGA-E2-A109-01,TCGA,I,val


In [6]:
pd.read_csv('/hpc/compgen/users/cchang/Projects/gene_exp_VAE/data/SyNet_fold_6.csv')

,sample_id,study_name,batch_label,cvset
0,GSM107072,Pawitan,F,train
1,GSM107073,Pawitan,F,train
2,GSM107074,Pawitan,F,train
3,GSM107075,Pawitan,F,train
4,GSM107076,Pawitan,F,train
...,...,...,...,...
3965,TCGA-E2-A108-01,TCGA,I,val
3966,TCGA-BH-A0B2-01,TCGA,I,val
3967,TCGA-AO-A12B-01,TCGA,I,val
3968,TCGA-E2-A109-01,TCGA,I,val


In [ ]:
df_batch_label.loc[df_batch_label.batch_label == 'I', 'cvset'] = 'test'
df_batch_label.loc[df_batch_label.batch_label == 'H', 'cvset'] = 'train'
# df_batch_label.loc[df_batch_label['batch_label'] == 'I']

df_batch_label.loc[df_batch_label.batch_label == 'J', 'cvset'] = 'val'
df_batch_label.loc[df_batch_label['study_name'] == 'WangY']

In [17]:
df_batch_label.to_csv('../data/SyNet_fold_7.csv')

In [151]:
df_batch_label['cvset'].value_counts()

train         3060
val            532
test           224
final_test     154
Name: cvset, dtype: int64

In [150]:
df_batch_label.to_csv('../data/SyNet_fold_6.csv', index=False)